In [13]:

import datetime
import json
import threading
import time

from pandas import Timestamp
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

In [14]:
import json
# load azure keys
with open('../keys/azure_keys.json') as f:
    azure_keys=json.load(f)
print(azure_keys)

{'stt_key1': 'key', 'openai_key1': 'dc0749bc213b4075ab8af52e2bdb442a'}


In [15]:
# import os
# from openai import AzureOpenAI
    
# client = AzureOpenAI(
#     api_key=azure_keys['openai_key1'],  
#     api_version="2024-02-01",
#     azure_endpoint = "https://genai-warriors-openai-test.openai.azure.com/"
#     )
    
# deployment_name='red-gpt' #This will correspond to the custom name you chose for your deployment when you deployed a model. Use a gpt-35-turbo-instruct deployment. 
    
# # Send a completion call to generate an answer
# # print('Sending a test completion job')
# start_phrase = 'Write a tagline for an ice cream shop. '
# response = client.completions.create(model=deployment_name, prompt=start_phrase, max_tokens=10)
# # print(start_phrase+response.choices[0].text)
# res=response.choices[0].text
# print(res)

In [16]:

def predict(context,question):
    print("-------------generating prediction-----------------")
    # data = json.loads(request.data)
    # data.context=data.context.replace('\n','\\n')
    # data.context=data.context.replace('\t','\\t')

    # context=request.args.get('context')
    # question=request.args.get('question')
    context=context.replace('\n','\\n')
    context=context.replace('\t','\\t')



    input=f'paragraph: {context}'
    instruction=f'Answer the following question from the meeting transcript: Question : {question}'
    # prompts=[[input,instruction]]
    prompt=f"""
    Answer the question from the meeting transcript in key points :
  
    #transcript start
    {context}
    #transcript end

    #question start
    {question}
    #question end

    """
    start_time = time.time()


    import os
    from openai import AzureOpenAI
    
    client = AzureOpenAI(
        api_key=azure_keys['openai_key1'],  
        api_version="2024-02-01",
        azure_endpoint = "https://genai-warriors-openai-test.openai.azure.com/"
        )
        
    deployment_name='red-gpt' #This will correspond to the custom name you chose for your deployment when you deployed a model. Use a gpt-35-turbo-instruct deployment. 
        
    # Send a completion call to generate an answer
    # print('Sending a test completion job')
    # start_phrase = 'Write a tagline for an ice cream shop. '
    response = client.completions.create(model=deployment_name, prompt=prompt, max_tokens=80)
    # print(start_phrase+response.choices[0].text)



    pt=time.time() - start_time
    print(f"--------------time taken by normal model={pt} seconds")
    res=response.choices[0].text
    print(res)

    return  { 'output': f'{res}' }


In [17]:

def listen_msgs():
    coll_name='sdlc_chat'
    user_uid='sdlc_chat_user'
        
    # Use a service account.
    cred = credentials.Certificate(f'../keys/sa.json')

    if not firebase_admin._apps:
       app = firebase_admin.initialize_app(cred)

    db = firestore.client()
    
    # Create an Event for notifying main thread.
    callback_done = threading.Event()
    global pdocid
    pdocid='000'
    # Create a callback on_snapshot function to capture changes
    def on_snapshot(doc_snapshot, changes, read_time) :
        global pdocid
        for doc in doc_snapshot:
            data=doc.to_dict()

            cdocid=data.get('timestamp').timestamp()
            if pdocid==cdocid or pdocid=='000':
                print("duplicate snapshot---------------------")
                pdocid=cdocid
                return
            else:
                print("new doc rec------------",cdocid,pdocid,type(cdocid),sep='\n')
                pdocid=cdocid



            print(f"Received document snapshot: {doc.id}")
            print(doc.to_dict())

            pred_res=predict(data['context'],data['message'])
            # pred_res={'output':"model response"}

            data['senderId']='chatbot@red'
            data['message']=pred_res['output']
            data['timestamp']=datetime.datetime.utcnow()
            print(f'sending response: '+data['message'])
            doc_id=str(round(time.time() * 1000))
            db.collection(coll_name).document(user_uid).collection('allMessages').document(doc_id).set(data)
            print(f"----------sent----------------with id: {doc_id} ")

        callback_done.set()

    doc_ref = db.collection(coll_name).document(user_uid).collection('userMessages').document("message")

    # Watch the document
    doc_watch = doc_ref.on_snapshot(on_snapshot)
    

    while True:
        print('', end='', flush=True)
        time.sleep(2)



In [18]:
try:
    print("listening for messages")
    listen_msgs()
except Exception as e:
    print(f'error occured:\n\n\n {e}')  

listening for messages
duplicate snapshot---------------------
new doc rec------------
1715676729.575
1715676662.997
<class 'float'>
Received document snapshot: message
{'senderId': 'sdlc_chat_user@red', 'rType': '1', 'userNotificationToken': '', 'message': 'what are the deadlines set in the meeting?', 'timestamp': DatetimeWithNanoseconds(2024, 5, 14, 8, 52, 9, 575000, tzinfo=datetime.timezone.utc), 'context': 'At Cognizant, we have been on a quest to advance AI beyond existing techniques, by taking a more holistic, human-centric approach to learning and understanding. As Chief Technology Officer of Azure AI Services, I have been working with a team of amazing scientists and engineers to turn this quest into a reality. In my role, I enjoy a unique perspective in viewing the relationship among three attributes of human cognition: monolingual text (X), audio or visual sensory signals, (Y) and multilingual (Z). At the intersection of all three, there’s magic—what we call XYZ-code as illus

KeyboardInterrupt: 